# 策略开发人员需要写的代码样例

In [1]:
#数据获取及历史数据存储
cps = dataApi.GetData('close')
old_alpha = np.empty(1)
hold_days = np.zeros(1)
start = True

industry = dataApi.GetData("industry") # accessing industry data
returns = dataApi.GetData('returns')

 
##重写虚函数
def SignalGenerate(di, alpha): 
  # Let your SignalGenerate() function know about global variables
  # Read-Only Global variables need not be declared here
  global start
  global old_alpha
  global hold_days
  # Initialize your Global variables once
  if start:

    old_alpha = np.empty(alpha.shape[0])

    hold_days = np.zeros(alpha.shape[0])

    start = False   

   

  # Calculate your new alpha value

  alpha[:] = (cps[di-delay-1,:] - cps[di-delay,:]) / cps[di-delay, :]

  alpha[:] = ss.rankdata(alpha[:]) / alpha.shape[0]

 

  # Hold your old alpha value, if hold_days < 10. Else use new alpha value

  alpha[:] = np.where(~np.isnan(old_alpha[:]) * hold_days[:] < 10, old_alpha[:], alpha[:])

  

  # Increment hold_days if holding old alpha value, else reset hold_days to 0

  hold_days[:] = np.where(~np.isnan(old_alpha[:]) * hold_days[:] < 10, hold_days[:] + 1, 0)

   

  # Save final alpha value for later use (next day)

  old_alpha[:] = alpha[:]

 

  # Keep only valid instruments, set rest to nan

  alpha[:] = np.where(valid[di, :], alpha[:], np.nan)

 

  # Create a grouping using returns

  group = factor_group(returns[di-delay, :], 10)

 

  # Neutralize your alpha against grouping from above

  alpha[:] = neutralize(group, alpha[:])

 

  # Neutralize your alpha against industry

  alpha[:] = neutralize(industry[di - delay, :], alpha[:])

 

 

def neutralize (group, alpha):

 

  # Get list of all unique groups

  group_uniq = np.unique(group)  

 

  # Iterate over each unique group 'g' in the list

  for g in group_uniq:

   

    # Skip if g is negative (invalid)

    if g < 0:

      continue

     

    # Filter out instruments which don't belong to group 'g'

    alpha_g = np.where(group[:] == g, alpha[:], np.nan)

    # OR

    #alpha_g = alpha [ group[:] == g ]

   

    # Calculate group mean for group 'g'

    group_mean = np.nanmean(alpha_g)

   

    # Subtract the group mean for all instruments belonging to group 'g'

    alpha[:] = np.where(group[:] == g, alpha[:] - group_mean, alpha[:])

 

  return alpha

 

def factor_group(factor, num_groups):

  # Rank the factor b/w 0 to 1

  factor_ranked = ss.rankdata(factor) / factor.shape[0]

 

  # Scale it: (0 to 1) --> (0 to num_groups), then convert to integer.

  group = np.floor(factor_ranked * num_groups)

 

  return group

NameError: name 'dataApi' is not defined